<a href="https://colab.research.google.com/github/MLandML/MLandML/blob/learning_projects/autoregressive_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PseudoCode:
given:
Wxh: input to hidden weight;
Whh: hidden to hidden weight;
bh: hidden bias;
Wo: hidden to output weight;
bo: output bias;
X - TxD input matrix

tanh hidden activation;
softmax output activation


```
Yhat = []
h_last = 0
for t in range(T):
    h_t = tanh(X[t].dot(Wxh)+ h_last.dot(Whh) + bh)
    yhat = sigmoid(h_t.dot(Wo) + bo)
    Yhat.append(yhat)
    h_last = h_t
```



In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.keras.layers import Input,SimpleRNN,Dense,Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD,Adam


In [ ]:
series = np.sin(0.1*np.arange(200)) + 0.1*np.random.randn(200)
plt.plot(series)

In [ ]:
T = 10
D = 1
X = []
Y = []

for t in range(len(series)-T):
  x = series[t:t+T]
  X.append(x)
  y = series[t+T]
  Y.append(y)

X = np.array(X).reshape(-1,T,D)
Y = np.array(Y)
N = len(X)

In [ ]:
Y.shape

In [ ]:
i = Input(shape = (T,D))
x = SimpleRNN(5,activation=None)(i) #default activation = tanh
x = Dense(1)(x)

model = Model(i,x)

model.compile(
    loss = 'mse',
    optimizer = Adam(lr=0.1)
)

r = model.fit(X[:-N//2],Y[:-N//2],validation_data=(X[-N//2:],Y[-N//2:]),epochs=100)

In [ ]:
plt.plot(r.history['loss'],label='loss')
plt.plot(r.history['val_loss'],label='val_loss')
plt.legend()

In [ ]:
val_targets = Y[-N//2:]
val_preds = []
last_x = X[-N//2]

while len(val_preds)<len(val_targets):
  p = model.predict(last_x.reshape(1,-1,1))[0,0]
  val_preds.append(p)
  last_x = np.roll(last_x,-1)
  last_x[-1] = p

In [ ]:
plt.plot(val_targets,label='forecasted_target')
plt.plot(val_preds,label='forecasted_prediction')
plt.legend()